In [1]:
import os
import sys
import time
import random
import warnings
import collections
from dateutil.relativedelta import relativedelta
from datetime import datetime
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

# Script to generate supplemental file for claim submission
Currently configured for UHC, other plans may be signifigantly different

In [2]:
# configuration
use_cache = False
seed = random.randint(0, 100)
from_date = '2022-03-23'

print(f'Seed: {seed}')

Seed: 44


### Pull Data

In [3]:
query = f"""
SELECT * FROM perm.supp_file_uhc_20230616;
"""

In [4]:
df = cb_utils.sql_query_to_df(query, use_cache=use_cache)

Pulling query from db


In [5]:
df.shape # 4609 -> 6954

(14204, 23)

In [6]:
df.mbi.nunique(),  df.member_id.nunique()

(10211, 10565)

In [7]:
df.head()

,elation_bill_id,patient_first_name,patient_last_name,patient_dob,mbi,member_id,gender,date_of_service,provider_first_name,provider_last_name,provider_email,provider_npi,medicare_specialty_code_int,tax_id,place_of_service,ra_code,line1,line2,city,state,postal_code,procedure_code,icds
0,550867007963292,Mamie,Webb,1938-01-27,4AN6GJ3MR42,106774307,F,2022-12-30,Mallory,Knott,mallory.knott@carebridgehealth.com,1477049864,50,84-2590508,10,A,270 Central Ave,None,Brownsville,TN,38012,99212,[L08.9]
1,566077201055900,Mamie,Webb,1938-01-27,4AN6GJ3MR42,106774307,F,2023-02-09,Lacey,Bolden,lacey.bolden@carebridgehealth.com,1740410703,50,84-2590508,10,A,270 Central Ave,None,Brownsville,TN,38012,99214,"[I20.8, I25.118, F32.3, H42, I73.9, E11.69, E7..."
2,499171401597084,Elizabeth,Jones,1947-04-02,2DY8W17YJ55,117029946,F,2022-08-09,Jessica,Flippo,jessica.flippo@carebridgehealth.com,1194221317,50,84-2590508,10,A,2833 Fawnridge Lane,None,Knoxville,TN,37938,99214,"[Z79.4, M06.9, E11.69, E11.69, G40.A09, I69.35..."
3,539730127880348,Elizabeth,Jones,1947-04-02,2DY8W17YJ55,117029946,F,2022-11-30,Mallory,Knott,mallory.knott@carebridgehealth.com,1477049864,50,84-2590508,10,A,2833 Fawnridge Lane,None,Knoxville,TN,37938,99212,"[R53.83, L30.9]"
4,572401843175580,Elizabeth,Jones,1947-04-02,2DY8W17YJ55,117029946,F,2023-02-27,Jessica,Flippo,jessica.flippo@carebridgehealth.com,1194221317,50,84-2590508,10,A,2833 Fawnridge Lane,None,Knoxville,TN,37938,99214,"[Z79.4, E11.69, E78.5, F33.41, G20, I69.352, M..."


In [8]:
# dupe check
assert df.elation_bill_id.nunique() == df.shape[0]

In [9]:
# icd check
assert df.loc[df.icds.isna()].shape[0] == 0

In [10]:
# if more than 40 we need to duplicate the row and add the additional icds on the dupe row
# currently not coded up because the situation doesn't exist
assert df.icds.apply(lambda x: x if x is None else len(set(x))).max() <= 40

#### Pad dx list col to 40

In [11]:
def pad_dx_col(icds):
    if icds is None:
        return [None] * 40
    
    icds = [i.replace('.', '') for i in set(icds)]
    
    if len(icds) < 40:
        return icds + [None] * (40 - len(icds))
    return icds
    
    
df.icds = df.icds.apply(pad_dx_col)

### Format dates

In [12]:
df.date_of_service = pd.to_datetime(df.date_of_service).dt.strftime('%m/%d/%Y')
df.patient_dob = pd.to_datetime(df.patient_dob).dt.strftime('%m/%d/%Y')

### Pad CMS IDs

In [13]:
df.place_of_service = df.place_of_service.str.pad(width=2, side='left', fillchar='0')
df.medicare_specialty_code_int = df.medicare_specialty_code_int.str.pad(width=2, side='left', fillchar='0')

### Build rows

In [14]:
def build_file_row(i, row):
    return [
        'DTL', # seg type
        row.elation_bill_id, # ref #
        row.patient_last_name,
        row.patient_first_name,
        None, # MI
        row.patient_dob,
        row.member_id if row.mbi is None else None, # member id
        None, # 'Retrieval NPI (Internal Use)',
        row.gender, # 'GENDER',
        None, # 'STATE CODE',
        row.mbi, # 'MBI-  need either member ID or MBI',
        row.date_of_service, # 'FDOS',
        row.date_of_service, # 'TDOS',
        None, # 'BILL TYPE- Institutional Only',
        None, # 'NU Indicator (Internal Use)',
        None, # 'PROV ID',
        row.provider_npi, # 'NPI',
        None, # 'PROV TYPE',
        None, # 'FACILITY NM- required for Institutional',
        row.provider_last_name, # 'PROV LAST NAME- Required for Professional',
        row.provider_first_name, # 'PROV FIRST NAME- required for Professional',
        f'{row.medicare_specialty_code_int}', # 'CMS SPECIALTY- Required for Professional',
        row.tax_id, # 'TAX ID',
        row.procedure_code, # 'CPT- Professional and Hospital Outpatient only',
        None, # 'REV CODE - Required for Institutional',
        None, # 'SERVICE FDOS',
        None, # 'SERVICE TDOS',
        row.place_of_service, # 'POS- Professional only',
        '0', # 'ICD INDIC',
        row.ra_code, # 'RA Code- Required for Professional',
        None, # 'Chart Barcode (Internal Use)',
        None, # 'Chart Enc Key (Internal Use)',
        None, # 'Chart DX Key  (Internal Use)',
        None, # 'Contract ID (Tufts use only)',
        row.line1, # 'Mem Street Address ',
        row.line2, # 'Mem Address 2',
        row.city, # 'Mem City',
        row.state, # 'Mem State',
        row.postal_code, # 'Mem Zip Code',
        None, # 'CLAIMID/PCN', -- This would link it to an existing claim and make it a resubmit
    ] + row.icds

rows = []
for i, row in df.iterrows():
    rows.append(build_file_row(i, row))

### Add header and footer rows

In [15]:
n_rows, n_columns = len(rows), len(rows[0])
# 100k limit per file
assert n_rows <= 100000

training_partner_id = '00795'
file_name = f'{training_partner_id}_UHC_ASMP_DIRECT_{datetime.now().strftime("%Y%m%d%H%M%S")}.txt'

header_vals = [
    'HDR',
    '8.9',
    training_partner_id,
    file_name,
    'UHC',
    'DIRECT',
    'P',
    'Y'
]
header_vals = header_vals + [None] * (n_columns - len(header_vals))
footer_vals = [
    'TRL',
    training_partner_id,
    n_rows
]
footer_vals = footer_vals + [None] * (n_columns - len(footer_vals))

In [16]:
rows.insert(0, header_vals)
rows.append(footer_vals)

### Build File

In [17]:
dx_cols = [f'DX_{i}' for i in range(40)]
columns = [
    'SEG TYPE',
    'REF #',
    'LAST NAME',
    'FIRST NAME',
    'MI',
    'DOB',
    'MEMBER ID- need either member ID or MBI',
    'Retrieval NPI (Internal Use)',
    'GENDER',
    'STATE CODE',
    'MBI-  need either member ID or MBI',
    'FDOS',
    'TDOS',
    'BILL TYPE- Institutional Only',
    'NU Indicator (Internal Use)',
    'PROV ID',
    'NPI',
    'PROV TYPE',
    'FACILITY NM- required for Institutional',
    'PROV LAST NAME- Required for Professional',
    'PROV FIRST NAME- required for Professional',
    'CMS SPECIALTY- Required for Professional',
    'TAX ID',
    'CPT- Professional and Hospital Outpatient only',
    'REV CODE - Required for Institutional',
    'SERVICE FDOS',
    'SERVICE TDOS',
    'POS- Professional only',
    'ICD INDIC',
    'RA Code- Required for Professional',
    'Chart Barcode (Internal Use)',
    'Chart Enc Key (Internal Use)',
    'Chart DX Key  (Internal Use)',
    'Contract ID (Tufts use only)',
    'Mem Street Address ',
    'Mem Address 2',
    'Mem City',
    'Mem State',
    'Mem Zip Code',
    'CLAIMID/PCN',
] + dx_cols

In [18]:
file_df = pd.DataFrame(rows, columns=columns)

In [19]:
file_df.head(20)

,SEG TYPE,REF #,LAST NAME,FIRST NAME,MI,DOB,MEMBER ID- need either member ID or MBI,Retrieval NPI (Internal Use),GENDER,STATE CODE,MBI- need either member ID or MBI,FDOS,TDOS,BILL TYPE- Institutional Only,NU Indicator (Internal Use),PROV ID,NPI,PROV TYPE,FACILITY NM- required for Institutional,PROV LAST NAME- Required for Professional,PROV FIRST NAME- required for Professional,CMS SPECIALTY- Required for Professional,TAX ID,CPT- Professional and Hospital Outpatient only,REV CODE - Required for Institutional,SERVICE FDOS,SERVICE TDOS,POS- Professional only,ICD INDIC,RA Code- Required for Professional,Chart Barcode (Internal Use),Chart Enc Key (Internal Use),Chart DX Key (Internal Use),Contract ID (Tufts use only),Mem Street Address,Mem Address 2,Mem City,Mem State,Mem Zip Code,CLAIMID/PCN,DX_0,DX_1,DX_2,DX_3,DX_4,DX_5,DX_6,DX_7,DX_8,DX_9,DX_10,DX_11,DX_12,DX_13,DX_14,DX_15,DX_16,DX_17,DX_18,DX_19,DX_20,DX_21,DX_22,DX_23,DX_24,DX_25,DX_26,DX_27,DX_28,DX_29,DX_30,DX_31,DX_32,DX_33,DX_34,DX_35,DX_36,DX_37,DX_38,DX_39
0,HDR,8.9,00795,00795_UHC_ASMP_DIRECT_20230616162512.txt,UHC,DIRECT,P,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,DTL,550867007963292,Webb,Mamie,None,01/27/1938,None,None,F,None,4AN6GJ3MR42,12/30/2022,12/30/2022,None,None,None,1477049864,None,None,Knott,Mallory,50,84-2590508,99212,None,None,None,10,0,A,None,None,None,None,270 Central Ave,None,Brownsville,TN,38012,None,L089,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,DTL,566077201055900,Webb,Mamie,None,01/27/1938,None,None,F,None,4AN6GJ3MR42,02/09/2023,02/09/2023,None,None,None,1740410703,None,None,Bolden,Lacey,50,84-2590508,99214,None,None,None,10,0,A,None,None,None,None,270 Central Ave,None,Brownsville,TN,38012,None,I739,N1831,E1169,I10,E785,I25118,F323,I208,H42,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,DTL,499171401597084,Jones,Elizabeth,None,04/02/1947,None,None,F,None,2DY8W17YJ55,08/09/2022,08/09/2022,None,None,None,1194221317,None,None,Flippo,Jessica,50,84-2590508,99214,None,None,None,10,0,A,None,None,None,None,2833 Fawnridge Lane,None,Knoxville,TN,37938,None,F3341,K219,E1169,M1990,G40A09,I10,E785,F0280,I69352,G20,G4700,M069,E559,Z794,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,DTL,539730127880348,Jones,Elizabeth,None,04/02/1947,None,None,F,None,2DY8W17YJ55,11/30/2022,11/30/2022,None,None,None,1477049864,None,None,Knott,Mallory,50,84-2590508,99212,None,None,None,10,0,A,None,None,None,None,2833 Fawnridge Lane,None,Knoxville,TN,37938,None,R5383,L309,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,DTL,572401843175580,Jones,Elizabeth,None,04/02/1947,None,None,F,None,2DY8W17YJ55,02/27/2023,02/27/2023,None,None,None,1194221317,None,None,Flippo,Jessica,50,84-2590508,99214,None,None,None,10,0,A,None,None,None,None,2833 Fawnridge Lane,None,Knoxville,TN,37938,None,F3341,F0390,K219,I739,E1169,M1990,G40A09,E1151,I10,E785,I69352,G20,M069,Z794,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,DTL,435902544674972,Simcox,Barbara,None,12/11/1943,None,None,F,None,1D74JP6NA76,04/11/2022,04/11/2022,None,None,None,1194221317,None,None,Flippo,Jessica,50,84-2590508,99214,None,None,None,11,0,A,None,Non

In [20]:
file_df.to_csv(f'../data/{file_name}', sep='|', index=False, header=False)

In [21]:
file_name

'00795_UHC_ASMP_DIRECT_20230616162512.txt'

### response

In [ ]:
import re

cols = [re.sub('[^0-9a-zA-Z]+', '_', c.lower()) for c in columns+['err_id', 'error']]
df = pd.read_csv('/Users/bp/workspace/cb/data/00795_UHC_ASMP_DIRECT_20220328162146_RESUB1.txt',
                 sep='|', header=None, skipfooter=1, skiprows=1, names=cols, engine='python'
                )
df.head()


In [ ]:
df.error.value_counts()

In [ ]:
# HDR|8.9|00795|00795_UHC_ASMP_DIRECT_20220328162146_RESUB1.txt|UHC|DIRECT|P|Y

In [ ]:
df.loc[df.error == 'err18-Blank MemberID; err40-Member not found in plan membership;']

In [1]:
df.to_csv('/Users/bp/Downloads/uhc_supp_file_errors_20230615.csv', index=False)

NameError: name 'df' is not defined